In [ ]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd

In [ ]:
from tiled.client import from_uri
client = from_uri("http://localhost:8000/api")

In [ ]:

import os
from pathlib import Path


DATA_PATH = Path("aimmdb/data/feff/65272_C_007")
print("Data Path:", DATA_PATH)

contents = os.listdir(DATA_PATH)
print("Contents:", contents)


In [ ]:
from load_FEFF_Data import load_feff_data

data, metadata = load_feff_data(DATA_PATH)

In [ ]:
def ingest_feff(client, df, verbose=False):
    """
    Upload the FEFF dataset to database
    """

    for (name, prep), g in df.groupby(["sample.name", "sample.prep"]):
        if verbose:
            print(f"{name}: {prep}, {len(g)}")

        sample_id = client.write_sample({"name" : name, "prep" : prep})

        for i, row in g.iterrows():
            feff_df, _ = read_dat(row.file)
            metadata = row.metadata
            metadata["dataset"] = "feff"
            metadata["sample_id"] = sample_id
            client["uid"].write_dataframe(feff_df, metadata=metadata, specs=["FEFF"])

In [ ]:
print("starting ingestion...")
ingest_feff(client, feff, verbose=True)
print("finished.")

In [ ]:
FEFFInputMetadata.schema()

In [ ]:
FEFFOutputMetadata.schema()

In [ ]:
# we will enforce that XAS metadata satisfies the following schema
ExperimentalXASMetadata.schema()

In [ ]:
client["uid"]

In [ ]:
# with the correct metadata we can write to the server
# NOTE this doesn't prevent you from writing garbage but does help
df = pd.DataFrame({"a" : np.random.rand(100), "b" : np.random.rand(100)})
metadata = {"dataset" : "feff", "foo" : "bar", "element" : {"symbol" : "Au", "edge" : "K"}, "facility" : {"name" : "ALS"}, "beamline" : {"name" : "8.0.1"}}
node = client["uid"].write_dataframe(df, metadata=metadata, specs=["FEFF"])
node